<a href="https://colab.research.google.com/github/yulia1309/visualprogram/blob/main/%D0%BB%D0%B0%D0%B12_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript

# Функция для запуска JavaScript, который захватывает видео с камеры
def start_video_capture():
    display(Javascript('''
        async function startVideo() {
            const video = document.createElement('video');
            video.width = 640;
            video.height = 480;
            video.autoplay = true;
            video.style.display = 'block';
            document.body.appendChild(video);
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            return video;
        }
        startVideo().then(video => {
            window.video = video;
        });
    '''))

# Запускаем видеозахват
start_video_capture()

# Открываем видеопоток с камеры
cap = cv2.VideoCapture(0)

# Получаем параметры видео
fps = 30  # Устанавливаем фиксированное значение FPS
width = 640
height = 480

# Создаем VideoWriter для сохранения результата (опционально)
out = cv2.VideoWriter('output_with_red_ball.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

frame_count = 0  # Счетчик кадров

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Обрабатываем каждый третий кадр
    if frame_count % 3 == 0:
        # Преобразуем кадр в цветовое пространство HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Определяем диапазоны красного цвета в HSV
        lower_red1 = np.array([0, 100, 100])    # Низкий диапазон красного
        upper_red1 = np.array([10, 255, 255])   # Высокий диапазон красного
        lower_red2 = np.array([170, 100, 100])  # Низкий диапазон красного (второй диапазон)
        upper_red2 = np.array([180, 255, 255])  # Высокий диапазон красного (второй диапазон)

        # Создаем маски для красного цвета
        mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
        mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
        mask = mask1 | mask2  # Объединяем маски

        # Находим контуры на маске
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            # Находим внешний контур
            largest_contour = max(contours, key=cv2.contourArea)

            # Рисуем контур на оригинальном кадре
            cv2.drawContours(frame, [largest_contour], -1, (0, 255, 0), 2)  # Рисуем контур зеленым цветом

            # Вычисляем моменты для нахождения центра масс
            M = cv2.moments(largest_contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                # Рисуем центр масс
                cv2.circle(frame, (cX, cY), 5, (255, 0, 0), -1)  # Рисуем центр масс синим цветом

                # Отображаем координаты центра масс
                cv2.putText(frame, f'Center: ({cX}, {cY})', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            else:
                cv2.putText(frame, 'No ball detected', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        else:
            cv2.putText(frame, 'No ball detected', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Сохраняем кадр с контурами в новый видеофайл (опционально)
        out.write(frame)

        # Отображаем кадр с контурами
        cv2_imshow(frame)

        # Пауза для отображения
        cv2.waitKey(1)

    frame_count += 1  # Увеличиваем счетчик кадров

# Освобождаем ресурсы
cap.release()
out.release()
cv2.destroyAllWindows()




<IPython.core.display.Javascript object>